In [1]:
# importing all important libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Wave (y) grid in Angstroms
gridy = np.linspace(3500, 8300, int((8300-3500)/10+1), dtype=int) # in Angstroms

%run ./code/make_surface.ipynb
%matplotlib inline

In [2]:
# Instantiating our python class. Here we are going to define our grid of time x wavelength
make = make_surface('exp2022', gridx=(-8, 48), gridy=(3500, 8300))

In [3]:
# Sending our data path. Modify the last part to your data path if you are using a different one
file_names = make.get_spec_path(sp_data_path='./data/sp/')

Total files in path:  2466


In [4]:
# A catalog file which consist of sn name, a given phase in days and its path
catalog = pd.read_csv('./data/list.dat', names=['name', 'phase', 'path'], sep='\s+')
catalog.head()

,name,phase,path
0,Test_SN0,-8.154,Test_SN0_0.dat
1,Test_SN0,-3.522,Test_SN0_1.dat
2,Test_SN0,17.959,Test_SN0_10.dat
3,Test_SN0,19.801,Test_SN0_11.dat
4,Test_SN0,19.816,Test_SN0_12.dat


In [5]:
# Defining our unique names, here is a sample of this pandas' series
training_sn = pd.Series(catalog['name'].unique())
training_sn.head()

0     Test_SN0
1    Test_SN10
2    Test_SN11
3    Test_SN12
4    Test_SN13
dtype: object

In [6]:
# retirar essa parte na versao final, minha ideia é fazer um nb só para os gráficos que entram no artigo
'''
loglike_gpr = []
gpr_methods = ['rbf', 'matern', 'rq']
for i in range(3):
    loglike_gpr.append([])

training_sn0 = catalog[catalog['name']=='Test_SN0']
training_sn0.reset_index(drop=True, inplace=True)
sn_data = []
for i in range(0, len(training_sn0)):
    
    spec = make.read_spec(spec_path='./data/sp/'+training_sn0['path'][i])
    smooth_flux = make.apply_filter(np.array(spec["SN_SPEC"]))
    function = make.salt2likeinterp(x_array=spec["WAVE"], y_array=smooth_flux, k=2)
    flux_reg = function(gridy)
    sn_data.append(flux_reg)
    
sn_data = np.array(sn_data).T

for j in range(0 ,len(loglike_gpr)):
    print(gpr_methods[j])
    for i in range(0, len(sn_data)):
        y_gauss, cov_gauss, loglike = make.apply_gp(x_array=training_sn0['phase'], y_array=sn_data[i], method=gpr_methods[j], returncov=True)
        loglike_gpr[j].append(loglike)
        #data[0].append(y_gauss)
        #data_error[0].append(cov_gauss)
'''

'\nloglike_gpr = []\ngpr_methods = [\'rbf\', \'matern\', \'rq\']\nfor i in range(3):\n    loglike_gpr.append([])\n\ntraining_sn0 = catalog[catalog[\'name\']==\'Test_SN0\']\ntraining_sn0.reset_index(drop=True, inplace=True)\nsn_data = []\nfor i in range(0, len(training_sn0)):\n    \n    spec = make.read_spec(spec_path=\'./data/sp/\'+training_sn0[\'path\'][i])\n    smooth_flux = make.apply_filter(np.array(spec["SN_SPEC"]))\n    function = make.salt2likeinterp(x_array=spec["WAVE"], y_array=smooth_flux, k=2)\n    flux_reg = function(gridy)\n    sn_data.append(flux_reg)\n    \nsn_data = np.array(sn_data).T\n\nfor j in range(0 ,len(loglike_gpr)):\n    print(gpr_methods[j])\n    for i in range(0, len(sn_data)):\n        y_gauss, cov_gauss, loglike = make.apply_gp(x_array=training_sn0[\'phase\'], y_array=sn_data[i], method=gpr_methods[j], returncov=True)\n        loglike_gpr[j].append(loglike)\n        #data[0].append(y_gauss)\n        #data_error[0].append(cov_gauss)\n'

In [7]:
'''
plt.hist(loglike_gpr[0], density=True, label='RBF', histtype='step', linewidth=3.)
plt.hist(loglike_gpr[1], density=True, label='Matern', histtype='step', linewidth=3.)
plt.hist(loglike_gpr[2], density=True, label='RQ', histtype='step', linewidth=3.)

plt.legend(loc='upper left')
plt.xlabel('log-likelihood')
plt.ylabel('f');
'''

"\nplt.hist(loglike_gpr[0], density=True, label='RBF', histtype='step', linewidth=3.)\nplt.hist(loglike_gpr[1], density=True, label='Matern', histtype='step', linewidth=3.)\nplt.hist(loglike_gpr[2], density=True, label='RQ', histtype='step', linewidth=3.)\n\nplt.legend(loc='upper left')\nplt.xlabel('log-likelihood')\nplt.ylabel('f');\n"

In [8]:
#np.mean(loglike_gpr[0])

In [9]:
#np.mean(loglike_gpr[1])

In [10]:
#np.mean(loglike_gpr[2])

In [11]:
# Our main cell. Here we are going to train our SEDs and write in .dat files
print('Starting to appply a filter and gaussian process to build SEDs. It may take some minutes, so fell free to take a break :)')
# Defining our data and data error lists
data = []
data_error = []
for i in range(0, len(training_sn)):
    data.append([])
    data_error.append([])
# Exceptions made by criteria
exceptions = []
# Python Loop over the sns (unique names obtained before)
for n in range(0, len(training_sn)):
    print(n)
    # Getting all file names that correspond to that SN
    training_sn_i = catalog[catalog['name']==training_sn[n]]
    training_sn_i = training_sn_i[(training_sn_i['phase']<48) & (training_sn_i['phase']>-8)]
    training_sn_i.reset_index(drop=True, inplace=True)
    # Lenght criteria
    if len(training_sn_i) < 7:
        exceptions.append(n)
    # Make a loop to regularize in the wavelength grid and apply our low-bandwidth filter
    
    sn_data = []
    for i in range(0, len(training_sn_i)):
        # Reading that spectra
        spec = make.read_spec(spec_path='./data/sp/'+training_sn_i['path'][i])
        # Applying a filter
        smooth_flux = make.apply_filter(np.array(spec["SN_SPEC"]))
        # Interpolating to regularization, here a cubic scipy spline
        function = make.salt2likeinterp(x_array=spec["WAVE"], y_array=smooth_flux, k=1)
        flux_reg = function(gridy)
        # Store in lists
        sn_data.append(flux_reg)
    # Transpose in order to match to our final dataframe
    sn_data = np.array(sn_data).T
    for i in range(0, len(sn_data)):
        # Finally applying our gaussian process
        y_gauss, cov_gauss, loglike = make.apply_gp(x_array=training_sn_i['phase'], y_array=sn_data[i], method='matern', returncov=True)
        # saving fluxes and given erros obtained from covariance in the fit
        data[n].append(y_gauss)
        data_error[n].append(cov_gauss)
# Export path
export_path = './SEDs_matern/'
# Storing our SEDs
for i in range(0, len(data)):
    if i not in exceptions:
        # Here we chose apply a np flatten to transform our 2d data into a 1d one. It takes more easy to work with
        data_i_flatten = np.array(data[i]).flatten()
        data_i_err_flatten = np.array(data_error[i]).flatten()
        # SED file name
        f = open(export_path+str(i)+'.dat', 'w')
        f.write('flux,fluxerr\n')
        for j in range(0, len(data_i_flatten)):
            # Writing data and error, side by side
            f.write(str(data_i_flatten[j])+', '+str(data_i_err_flatten[j])+'\n')
        f.close()
        
print("Done! Your SEDs are ready to the train process.")

Starting to appply a filter and gaussian process to build SEDs. It may take some minutes, so fell free to take a break :)
0
1
2


/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k1__length_scale is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k1__length_scale is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k1__length_scale is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_p

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:610: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k1__length_scale is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k1__length_scale is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


19
20
21


...

/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k1__length_scale is close to the specified lower bound 1.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k2__noise_level is close to the specified upper bound 1.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k1__length_scale is close to the specified lower bound 1.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/ker

/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k1__length_scale is close to the specified lower bound 1.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k2__noise_level is close to the specified upper bound 1.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k1__length_scale is close to the specified lower bound 1.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/ker

/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k2__noise_level is close to the specified upper bound 1.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k2__noise_level is close to the specified upper bound 1.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k2__noise_level is close to the specified upper bound 1.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/kerne

/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k2__noise_level is close to the specified upper bound 1.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k2__noise_level is close to the specified upper bound 1.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k2__noise_level is close to the specified upper bound 1.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/kerne

Done! Your SEDs are ready to the train process.
